In [29]:
import pandas as pd

In [30]:
# ( employee_id,  firstname ,  lastname ,  birthdate ,  sex ,  street ,  city ,  zip ,  country ,  emailaddress ,  telephoneno ,  salary ,  department ,  username ,  password ) 
employee = open('employee_raw.txt', 'r')
employees = []
for line in employee:
    employees.append(line.split(',')[15:-2])
employees = pd.DataFrame(employees, columns=[ 'firstname' ,  'lastname' ,  'birthdate' ,  'sex' ,  'street' ,  'city' ,  'zip' ,  'country' ,  'emailaddress' ,  'telephoneno' ,  'salary' , 'department', 'none'])
employees = employees[pd.isna(employees['none'])]
employees = employees.drop(columns=['birthdate', 'sex', 'street', 'city', 'zip', 'emailaddress', 'none'])
employees['country_name'] = employees['country'].str.strip("'")

In [31]:
# count number of employees in each dept
dept_count = employees['department'].value_counts()
print(dept_count)

# rename departments
employees['department'] = employees['department'].str.replace("'Marketing'", "'Flight Attendant'")
employees['department'] = employees['department'].str.replace("'Accounting'", "'Air Traffic Control'")
employees['department'] = employees['department'].str.replace("'Management'", "'Gate Agent'")
employees['department'] = employees['department'].str.replace("'Logistics'", "'Pilot'")

# count number of employees in each dept
dept_count = employees['department'].value_counts()
print(dept_count)


department
'Airfield'      205
'Marketing'     197
'Logistics'     193
'Management'    182
'Accounting'    175
Name: count, dtype: int64
department
'Airfield'               205
'Flight Attendant'       197
'Pilot'                  193
'Gate Agent'             182
'Air Traffic Control'    175
Name: count, dtype: int64


In [32]:
# If department is "'Pilot'", create fake pilot license number from phone number
for index, row in employees.iterrows():
    if row['department'] == "'Pilot'":
        employees.at[index, 'pilot_license'] = row['telephoneno'].replace(' ', '').replace("'", "")[4:-1]
    else:
        employees.at[index, 'pilot_license'] = None

In [33]:
airlines = pd.read_csv('airlines.csv')
airlines

,IATA,Airline,Country_Region
0,AA,American Airlines,United States
1,AC,Air Canada,Canada
2,AD,Azul Linhas Aéreas Brasileiras,Brazil
3,AE,Mandarin Airlines,Taiwan
4,AF,Air France,France
...,...,...,...
567,ZS,Sama Airlines,Saudi Arabia
568,ZT,Titan Airways,United Kingdom
569,ZW,Air Wisconsin,United States
570,ZX,Air Georgian,Canada


In [34]:
import random

# Give a 50 percent chance of each employee being from the United States
def random_country(country):
    if country == 'United States':
        return country
    else:
        return random.choice(['United States', 'United States', country])

employees['country_name'] = employees['country_name'].apply(random_country)

In [35]:

# Function to get a random airline based on country
def get_random_airline(country):
    # If country is USA, give a 70% chance of returning a random US airline
    if country == 'United States' and random.random() < 0.7:
        return random.choice(['DL', 'AA', 'WN', 'UA'])
    matching_airlines = airlines[airlines['Country_Region'] == country]
    if not matching_airlines.empty:
        return random.choice(matching_airlines['IATA'].values)
    else:
        return random.choice(airlines['IATA'].values)

# Apply the function to each employee
employees['airline_id'] = employees['country_name'].apply(get_random_airline)

# Remove employees with no matching airline
employees = employees.dropna(subset=['airline_id'])

# Display the updated dataframe
employees.head()

,firstname,lastname,country,telephoneno,salary,department,country_name,pilot_license,airline_id
0,'Michael','Ryo','Spain','04055 81801638',1431.93,'Air Traffic Control',United States,None,WN
1,'Rob','Zettler','United States','04131 97082561',1781.48,'Air Traffic Control',United States,None,DL
2,'David','Thomas','Algeria','08726 4534450',1960.80,'Pilot',Algeria,6453445,AH
3,'Lance','Leslie','Ethiopia','0308 38840119',1224.95,'Pilot',Ethiopia,3884011,ET
4,'Jonathan Taylor','Thomas','Korea','01377 38233600',1624.84,'Airfield',Korea,None,AH


In [36]:
airports = pd.read_csv('airports_merged.csv')

# Function to get a random airline based on country
def get_random_airport(country):
    # If country is United States, give a 70% chance of returning a large US airport
    if country == 'United States' and random.random() < 0.7:
        return random.choice(['ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'DEN', 'SFO', 'SEA', 'LAS', 'MCO', 'EWR', 'CLT', 'PHX', 'IAH', 'MIA', 'MSP', 'BOS', 'DTW', 'PHL', 'LGA'])
    matching_airlines = airports[airports['country_name'] == country]
    if not matching_airlines.empty:
        return random.choice(matching_airlines['code'].values)
    else:
        return random.choice(airports['code'].values)

# Apply the function to each employee
employees['code'] = employees['country_name'].apply(get_random_airport)

# Remove employees with no matching airline
employees = employees.dropna(subset=['code'])

# Display the updated dataframe
employees.head()

,firstname,lastname,country,telephoneno,salary,department,country_name,pilot_license,airline_id,code
0,'Michael','Ryo','Spain','04055 81801638',1431.93,'Air Traffic Control',United States,None,WN,IRK
1,'Rob','Zettler','United States','04131 97082561',1781.48,'Air Traffic Control',United States,None,DL,MRF
2,'David','Thomas','Algeria','08726 4534450',1960.80,'Pilot',Algeria,6453445,AH,GJL
3,'Lance','Leslie','Ethiopia','0308 38840119',1224.95,'Pilot',Ethiopia,3884011,ET,DEM
4,'Jonathan Taylor','Thomas','Korea','01377 38233600',1624.84,'Airfield',Korea,None,AH,TEO


In [37]:
# Export the updated dataframe to a new CSV file
employees.to_csv('employees.csv', index=False)

In [38]:

# Format each row for SQL entry
formatted_rows = []
for index, row in employees.iterrows():
    if pd.isna(row['pilot_license']):
        license = 'NULL'
    else:
        license = f"{row['pilot_license']}"
    formatted_row = f"INSERT INTO crew (f_name, l_name, role, license_no, airline_id, salary, code) VALUES({row['firstname']}, {row['lastname']}, {row['department']}, {license}, '{row['airline_id']}', {row['salary']}, '{row['code']}')"
    formatted_rows.append(formatted_row)

# Join all formatted rows into a single string
formatted_text = ';\n'.join(formatted_rows)

# Export the formatted text to a plaintext file
with open('Crew_Code_SQL_Entries.txt', 'w') as file:
    file.write(formatted_text)